In [8]:
import pandas as pd

In [9]:
produtos = pd.read_excel('tiny_produtos.xls')
kit = pd.read_excel('tiny_kit.xls')

excel = 'Planilha - Caroline Santos.xlsx' #  TROCAR NOME DA PLANILHA
base = pd.ExcelFile(excel)

base_produtos = base.parse('PRODUTOS', header=1)
base_kit = base.parse('COMPONENTES', header=1)


In [10]:
def preencher(id=None, sku=None, descricao=None, unidade=None, ncm=None, origem=None, preco=None, preco_custo=None, gtin=None, tipo=None, cod_pai=None, variacao=None):
    return {
        'ID': id,
        'Código (SKU)': sku,
        'Descrição': descricao,
        'Unidade': unidade,
        'NCM (Classificação fiscal)': ncm,
        'Origem': origem,
        'Preço': preco,
        'Preço de custo': preco_custo,
        'Situação': 'Ativo',
        'GTIN/EAN': gtin,
        'Tipo do produto': tipo,
        'Código do pai': cod_pai,
        'Variações': variacao,
        'Sob encomenda': 'Não',
        'Controlar lotes': 'Não'
    }

def montar_kit(sku, descricao):
    lista = list()
    linhas_filtradas = base_kit.loc[base_kit['Código (SKU)'] == sku, 'Linha'].values
    if len(linhas_filtradas) > 0:
        indice_linha = linhas_filtradas[0]
        linhas = base_kit['Linha'].tolist()

        while True:
            indice_linha += 1
            if indice_linha not in linhas:
                return lista
            tipo, sku_componente, quantidade = base_kit.loc[base_kit['Linha'] == indice_linha, ['Tipo do produto', 'Código (SKU)', 'Quantidade']].values[0]
            descricao_componente = base_produtos.loc[base_produtos['SKU'] == sku_componente, 'Nome'].values[0]
            if tipo == 'COMPONENTE':
                lista.append({
                    'ID kit/fabricado': None,
                    'SKU kit/fabricado': sku,
                    'Descrição kit/fabricado': descricao,
                    'ID componente': None,
                    'SKU componente': sku_componente,
                    'Descrição componente': descricao_componente,
                    'Quantidade componente': quantidade
                })
            else:
                return lista
    else:
        return lista


In [11]:
dados = list()
lista = list()
sku_pai = None
descricao_pai = None

for indice, linha in base_produtos.iterrows():
    sku = linha['SKU']
    descricao = linha['Nome']
    unidade = linha['UN']
    ncm = linha['NCM']
    origem = linha['Origem']
    preco = linha['Preço de Venda']
    custo = linha['Preço de custo']
    ean = linha['GTIN/EAN']

    if linha['Tipo do produto'] == 'SIMPLES':
        resultado = preencher(sku=sku, descricao=descricao, unidade=unidade, ncm=ncm, origem=origem, preco=preco, preco_custo=custo, gtin=ean, tipo='S')
        dados.append(resultado)

    elif linha['Tipo do produto'] == 'VARIAÇÃO PAI':
        sku_pai = linha['SKU']
        descricao_pai = linha['Nome']

        resultado = preencher(sku=sku, descricao=descricao, unidade=unidade, ncm=ncm, origem=origem, preco=preco, preco_custo=custo, gtin=ean, tipo='V')
        dados.append(resultado)

    elif linha['Tipo do produto'] == 'VARIAÇÃO':
        resultado = preencher(sku=sku, descricao=descricao_pai, unidade=unidade, ncm=ncm, origem=origem, preco=preco, preco_custo=custo, gtin=ean, tipo='S', cod_pai=sku_pai, variacao=descricao)
        dados.append(resultado)

    elif linha['Tipo do produto'] == 'KIT':
        resultado = preencher(sku=sku, descricao=descricao, unidade='KIT', ncm=ncm, origem=origem, preco=preco, preco_custo=custo, gtin=ean, tipo='K')
        dados.append(resultado)
        
        info = montar_kit(sku, descricao)
        lista.extend(info)

    elif linha['Tipo do produto'] == 'VARIAÇÃO KIT':
        resultado = preencher(sku=sku, descricao=descricao_pai, unidade='KIT', ncm=ncm, origem=origem, preco=preco, preco_custo=custo, gtin=ean, tipo='K', cod_pai=sku_pai, variacao=descricao)
        dados.append(resultado)

        info = montar_kit(sku, descricao)
        lista.extend(info)

    else:
        print(f'O produto {linha["Nome"]} não foi inserido na tabela!')


In [12]:
tabela = pd.DataFrame(dados)
tiny_produtos = pd.concat([produtos, tabela])
tiny_produtos.to_excel('Tabela CAROL.xlsx', index=False) #  MUDAR NOME DA PLANILHA DE PRODUTOS

if lista:
    kits = pd.DataFrame(lista)
    tiny_kit = pd.concat([kit, kits])
    tiny_kit.to_excel('Tabela Kits MARCOS.xlsx', index=False) #  MUDAR NOME DA PLANILHA DE KITS


C:\Users\eduar\AppData\Local\Temp\ipykernel_14432\559505664.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tiny_produtos = pd.concat([produtos, tabela])
